# 1. Define Data Scope - Comics
## - image is .jpg
## - book only has one author

In [2]:
import pandas as pd

In [1]:


df_books = pd.read_json('goodreads_books_comics_graphic.json', lines=True)
df_authors = pd.read_csv('author_dataset.csv')
df_reviews = pd.read_json('goodreads_reviews_comics_graphic.json', lines=True)
df_interactions = pd.read_json('goodreads_interactions_comics_graphic.json', lines=True)

MemoryError: 

In [6]:
df_books['image_type'] = df_books.image_url.apply(lambda x: 'jpg' if 'jpg' in x else 'png')
df_authors['image_type'] = df_authors.image_url.apply(lambda x: 'jpg' if 'jpg' in x else 'png')

df_books['author_count'] = df_books.authors.apply(lambda x: len(x))

In [29]:
print("Books: ")
print(df_books.image_type.value_counts())
print(" ")
print(df_books.image_type.value_counts(normalize=True))
print("-"*50)
print("Authors: ")
print(df_authors.image_type.value_counts())
print(" ")
print(df_authors.image_type.value_counts(normalize=True))

Books: 
jpg    57394
png    32017
Name: image_type, dtype: int64
 
jpg    0.641912
png    0.358088
Name: image_type, dtype: float64
--------------------------------------------------
Authors: 
png    125393
jpg     84124
Name: image_type, dtype: int64
 
png    0.598486
jpg    0.401514
Name: image_type, dtype: float64


In [37]:
df_books_filtered = df_books[(df_books.image_type=='jpg')&(df_books.author_count==1)][['book_id','authors']]

df_books_filtered['author_id'] = df_books_filtered.authors.apply(lambda x: int(x[0]['author_id']))

df_authors_filtered = df_authors[(df_authors.image_type=='jpg')][['authorid','name']]

df_final = df_books_filtered.merge(df_authors_filtered, how='inner', left_on='author_id', right_on='authorid')

In [67]:
books_list = df_final.book_id.unique().tolist()
authors_list = df_final.authorid.unique().tolist()

In [70]:
df_reviews_filter = df_reviews[df_reviews.book_id.isin(books_list)]

df_interactions_filter = df_interactions[df_interactions.book_id.isin(books_list)]

df_books_filter = df_books[df_books.book_id.isin(books_list)]

df_author_filter = df_authors[df_authors.authorid.isin(authors_list)]

In [87]:
print("-"*50)
print("Original Data Shape:")
print(" - books: ", df_books.shape)
print(" - authors:", df_authors.shape)
print(" - reviews:", df_reviews.shape)
print(" - interactions:", df_interactions.shape)
print("-"*50)
print("Data Shape after filtering:")
print(" - books: ", df_books_filter.shape)
print(" - authors:", df_author_filter.shape)
print(" - reviews:", df_reviews_filter.shape)
print(" - interactions:", df_interactions_filter.shape)
print("-"*50)

--------------------------------------------------
Original Data Shape:
 - books:  (89411, 31)
 - authors: (209517, 21)
 - reviews: (542338, 11)
 - interactions: (7347630, 10)
--------------------------------------------------
Data Shape after filtering:
 - books:  (16288, 31)
 - authors: (1920, 21)
 - reviews: (108323, 11)
 - interactions: (1426160, 10)
--------------------------------------------------


In [89]:
df_books_filter.to_csv('books.csv', index=False)
df_author_filter.to_csv('authors.csv', index=False)
df_reviews_filter.to_csv('reviews.csv', index=False)
df_interactions_filter.to_csv('interactions.csv', index=False)

# 2. Data Processing

In [3]:
df_books = pd.read_csv('books.csv')
df_authors = pd.read_csv('authors.csv')
df_reviews = pd.read_csv('reviews.csv')
df_interactions = pd.read_csv('interactions.csv')

In [4]:
list_df = [df_books, df_authors, df_reviews, df_interactions]
list_dataset_name = ['books', 'authors', 'reviews', 'interactions']

for i in range(len(list_df)):
    print(list_dataset_name[i]+": ")
    print(list_df[i].columns.tolist())
    print("")

books: 
['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code', 'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin', 'similar_books', 'description', 'format', 'link', 'authors', 'publisher', 'num_pages', 'publication_day', 'isbn13', 'publication_month', 'edition_information', 'publication_year', 'url', 'image_url', 'book_id', 'ratings_count', 'work_id', 'title', 'title_without_series', 'image_type', 'author_count']

authors: 
['authorid', 'name', 'workcount', 'fan_count', 'gender', 'image_url', 'about', 'born', 'died', 'influence', 'average_rate', 'rating_count', 'review_count', 'website', 'twitter', 'genre', 'original_hometown', 'country', 'latitude', 'longitude', 'image_type']

reviews: 
['user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']

interactions: 
['user_id', 'book_id', 'review_id', 'is_read', 'rating', 'review_text_incomplete', 'date_added', 'date_upda

## 2.1 remove unrelated columns

In [6]:
list_books_col = ['book_id', 'isbn', 'country_code', 'language_code', 'popular_shelves', 'is_ebook', 'average_rating',
                  'similar_books', 'description', 'format', 'link', 'authors', 'publisher', 'num_pages', 
                  'edition_information', 'publication_year', 'image_url', 'text_reviews_count', 'ratings_count', 'title']
list_authors_col = ['authorid', 'name', 'workcount', 'fan_count', 'gender', 'image_url', 'about', 'born', 
                    'died', 'influence', 'average_rate', 'genre', 'original_hometown', 'country', 'latitude', 'longitude']
list_reviews_col = ['review_id', 'rating', 'review_text', 'date_added', 'n_votes']
list_interactions_col = ['review_id','is_read','rating','date_added']

In [56]:
df_books_col_filtered = df_books[list_books_col]
df_authors_col_filtered = df_authors[list_authors_col]
df_reviews_col_filtered = df_reviews[list_reviews_col]
df_interactions_col_filtered = df_interactions[list_interactions_col]

## 2.2 check the null percentage

In [14]:
list_df_col_filtered = [df_books_col_filtered, df_authors_col_filtered, df_reviews_col_filtered, df_interactions_col_filtered]
for i in range(len(list_df)):
    print('-'*50)
    print(list_dataset_name[i]+": ")
    print(list_df_col_filtered[i].isnull().sum() * 100 / len(list_df_col_filtered[i]))

--------------------------------------------------
books: 
book_id                 0.000000
isbn                   39.280452
country_code            0.000000
language_code          22.286346
popular_shelves         0.000000
is_ebook                0.000000
average_rating          0.000000
similar_books           0.000000
description            13.641945
format                 14.912819
link                    0.000000
authors                 0.000000
publisher              11.044941
num_pages              18.418468
edition_information    92.135314
publication_year       10.842338
image_url               0.000000
text_reviews_count      0.000000
ratings_count           0.000000
title                   0.000000
dtype: float64
--------------------------------------------------
authors: 
authorid              0.000000
name                  0.000000
workcount             0.000000
fan_count             0.000000
gender                0.000000
image_url             0.000000
about              

## 2.3 remove columns with high null percentage

In [57]:
df_books_col_filtered.drop(['edition_information'], axis=1, inplace=True)
df_authors_col_filtered.drop(['influence'], axis=1, inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## 2.4 implement data preprocessing
## - impute missing values
## - modify data format

### 2.4.1 Books Dataset

In [60]:
# fill null values
df_books_col_filtered.isbn.fillna(-1, inplace=True)
df_books_col_filtered.language_code.fillna('unknown', inplace=True)
df_books_col_filtered.description.fillna('unknown', inplace=True)
df_books_col_filtered.format.fillna('unknown', inplace=True)
df_books_col_filtered.publisher.fillna('unknown', inplace=True)
df_books_col_filtered.num_pages.fillna(-1, inplace=True)
df_books_col_filtered.publication_year.fillna(-1, inplace=True)

# convert data types of some columns
df_books_col_filtered.isbn = df_books_col_filtered.isbn.astype('str')
df_books_col_filtered.num_pages = df_books_col_filtered.num_pages.astype('int64').astype('str')
df_books_col_filtered.publication_year = df_books_col_filtered.publication_year.astype('int64').astype('str')

# replace -1 with unknown
df_books_col_filtered.isbn = df_books_col_filtered.isbn.apply(lambda x: 'unknown' if x=='-1' else x)
df_books_col_filtered.num_pages = df_books_col_filtered.num_pages.apply(lambda x: 'unknown' if x=='-1' else x)
df_books_col_filtered.publication_year = df_books_col_filtered.publication_year.apply(lambda x: 'unknown' if x=='-1' else x)

# extract author_id
df_books_col_filtered.authors = df_books_col_filtered.authors.apply(lambda x: x.replace('\'','').split(',')[0].split(': ')[1])

In [74]:
df_books_col_filtered.dtypes

book_id                 int64
isbn                   object
country_code           object
language_code          object
popular_shelves        object
is_ebook                 bool
average_rating        float64
similar_books          object
description            object
format                 object
link                   object
authors                object
publisher              object
num_pages              object
publication_year       object
image_url              object
text_reviews_count      int64
ratings_count           int64
title                  object
dtype: object

### 2.4.2 Authors Dataset

In [80]:
# fill null values
df_authors_col_filtered.about.fillna('unknown', inplace=True)
df_authors_col_filtered.born.fillna('unknown', inplace=True)
df_authors_col_filtered.died.fillna('unknown', inplace=True)
df_authors_col_filtered.genre.fillna('unknown', inplace=True)
df_authors_col_filtered.original_hometown.fillna('unknown', inplace=True)
df_authors_col_filtered.country.fillna('unknown', inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [82]:
df_authors_col_filtered.dtypes

authorid               int64
name                  object
workcount              int64
fan_count              int64
gender                object
image_url             object
about                 object
born                  object
died                  object
average_rate         float64
genre                 object
original_hometown     object
country               object
latitude             float64
longitude            float64
dtype: object

### 2.4.3 Reviews Dataset

In [88]:
# fill null values
df_reviews_col_filtered.review_text.fillna('unknown', inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [89]:
df_reviews_col_filtered.dtypes

review_id      object
rating          int64
review_text    object
date_added     object
n_votes         int64
dtype: object

# 3. Export the Final Data

In [93]:
print("Data Shape:")
print(" - books: ", df_books_col_filtered.shape)
print(" - authors: ", df_authors_col_filtered.shape)
print(" - reviews:", df_reviews_col_filtered.shape)
print(" - interactions:", df_interactions_col_filtered.shape)

Data Shape:
 - books:  (16288, 19)
 - authors:  (1920, 15)
 - reviews: (108323, 5)
 - interactions: (1426160, 4)


In [ ]:
for i in range(len(list_df)):
    print(list_dataset_name[i]+": ")
    print(list_df_col_filtered[i].isnull().sum() * 100 / len(list_df_col_filtered[i]))
    list_df_col_filtered[i].to_csv(list_dataset_name[i]+'_final_dateset.csv', index=False)